<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - Photovoltaic Farms Mapping

## How to use a pre-trained model for inference with onnxruntime in an openEO User Defined Function (UDF)

Firstly we import the required libraries and  establish a connection with the openEO back-end.

In [1]:
import openeo
from utils import preprocess_sentinel2_data, postprocess_inference_data
from urllib.parse import urljoin

# Connect and authenticate
conn = openeo.connect("https://openeo.cloud/").authenticate_oidc()
# Alternatively you can also use the Copernicus Data Space Ecosystem (CDSE)
# conn = openeo.connect("https://openeo.dataspace.copernicus.eu/").authenticate_oidc()

Authenticated using refresh token.


A workflow for processing Sentinel-2 Level-2A earth observation data using openEO in orden to obtain the desired data, compatible with the classifier.

In [2]:
spatial_extent={
            "east": 16.414,
            "north": 48.008,
            "south": 47.962,
            "west": 16.342
    },
temporal_extent = ["2023-05-01","2023-09-30"]

s2_cube = preprocess_sentinel2_data(conn, spatial_extent, temporal_extent)

Load in the model from the VITO artifactory and the required dependencies and excecute the prediction job.

The output is stored as a geoTIFF in the local directory.

In [ ]:
# Supply the model as a URL. The model is stored in artifactory
model_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/photovoltaic/rf_1_median_depth_15.onnx"
udf = openeo.UDF.from_file(
    "udf_rf_onnx.py", 
    context={
        "model_url": model_url
    }
)

dependencies_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/onnx_dependencies_1.16.3.zip"
dependencies_name = "onnx_deps"
full_url = urljoin(dependencies_url, f"#{dependencies_name}")

job_options = {
    "udf-dependency-archives": [
        full_url
    ],
}


# Reduce the bands dimesnion to a single prediction using the udf
prediction = s2_cube.reduce_bands(reducer=udf)

# Post process the result with opening operation (erosion + dilation)
prediction_filtered = postprocess_inference_data(prediction, 3) 

prediction_job = prediction_filtered.execute_batch(
    "./photovoltaic_prediction_median_summer_2023.tiff",
    job_options=job_options,
    title="photovoltaic_prediction_median_summer_2023"
)